In [1]:
import pandas as pd
from sklearn.datasets import load_iris

In [2]:
d = load_iris()

In [3]:
dir(d)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [4]:
df = pd.DataFrame(d.data, columns = d.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [5]:
df['Target'] = d.target
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [6]:
d.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [9]:
df['flower_name'] = df.Target.apply(lambda x: d.target_names[x])
df[45:50]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target,flower_name
45,4.8,3.0,1.4,0.3,0,setosa
46,5.1,3.8,1.6,0.2,0,setosa
47,4.6,3.2,1.4,0.2,0,setosa
48,5.3,3.7,1.5,0.2,0,setosa
49,5.0,3.3,1.4,0.2,0,setosa


In [10]:
from sklearn.model_selection import train_test_split

In [14]:
 x_train, x_test, y_train, y_test = train_test_split(d.data, d.target, test_size = 0.2)

In [15]:
from sklearn.svm import SVC

In [16]:
sv = SVC(C = 30, kernel='rbf', gamma='auto')

In [18]:
sv.fit(x_train, y_train)
sv.score(x_test, y_test)

0.9666666666666667

In [29]:
from sklearn.model_selection import cross_val_score
cv0 = cross_val_score(SVC(C = 30, kernel='rbf', gamma='auto'), d.data, d.target, cv = 5)

In [30]:
import numpy as np

In [31]:
np.average(cv0)
# np.mean(cv)

0.96

In [32]:
cv1 = cross_val_score(SVC(C = 20, kernel='linear', gamma='auto'), d.data, d.target, cv = 5)
np.average(cv1)

0.9666666666666666

In [33]:

cv2 = cross_val_score(SVC(C = 10, kernel='rbf', gamma='scale'), d.data, d.target, cv = 5)
np.average(cv2)

0.9800000000000001

## Value changed for each paratmeter
so we can make it easier like..

In [42]:
kernel = ['linear', 'rbf']
c = [1,10,20]
avg_score = {} # dictinary formate 
for i in kernel:
    for j in c:
        h = cross_val_score(SVC(kernel = i, C = j, gamma='scale'), d.data, d.target, cv = 5)
        avg_score[f'kernel: {i} and  C: {str(j)}'] = np.mean(h)
        
avg_score

{'kernel: linear and  C: 1': 0.9800000000000001,
 'kernel: linear and  C: 10': 0.9733333333333334,
 'kernel: linear and  C: 20': 0.9666666666666666,
 'kernel: rbf and  C: 1': 0.9666666666666666,
 'kernel: rbf and  C: 10': 0.9800000000000001,
 'kernel: rbf and  C: 20': 0.9800000000000001}

## This is easier than previous approch but still here you can face some issues like here i am using only 2 parameter that's why i use to for loops but if you want to work with 4 or 5 or more parameter then there is a strong possiblity to make mistake 

Here sklearn gives a easy solution . we can use GridSearchCV or RandomizedSearchCv function

In [43]:
from sklearn.model_selection import GridSearchCV

In [46]:
GS_cv = GridSearchCV(SVC(),{
    'C': [1,10,20],
    'kernel': ['rbf','linear'],
    'degree': [1,2,3],
    'gamma': ['scale','auto']
}, cv = 5, return_train_score = False)

GS_cv.fit(d.data, d.target)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 10, 20], 'degree': [1, 2, 3],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['rbf', 'linear']})

In [50]:
hr = pd.DataFrame(GS_cv.cv_results_)
hr[:2]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.0008,0.00040,0.0006,0.00049,1,1,scale,rbf,"{'C': 1, 'degree': 1, 'gamma': 'scale', 'kerne...",0.966667,0.966667,0.966667,0.933333,1.0,0.966667,0.021082,28
1,0.0006,0.00049,0.0006,0.00049,1,1,scale,linear,"{'C': 1, 'degree': 1, 'gamma': 'scale', 'kerne...",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1


In [52]:
hr1 = hr[['param_C','param_kernel','param_degree','param_gamma','mean_test_score']]

In [73]:
hr1['mean_test_score'].max()

0.9800000000000001

In [71]:
hr1[hr1['mean_test_score'] > 0.980000]

,param_C,param_kernel,param_degree,param_gamma,mean_test_score
1,1,linear,1,scale,0.98
2,1,rbf,1,auto,0.98
3,1,linear,1,auto,0.98
5,1,linear,2,scale,0.98
6,1,rbf,2,auto,0.98
7,1,linear,2,auto,0.98
9,1,linear,3,scale,0.98
10,1,rbf,3,auto,0.98
11,1,linear,3,auto,0.98
12,10,rbf,1,scale,0.98


In [74]:
dir(GS_cv)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 '

In [75]:
GS_cv.best_score_

0.9800000000000001

In [76]:
hr1[hr1['mean_test_score'] == 0.9800000000000001]

,param_C,param_kernel,param_degree,param_gamma,mean_test_score
1,1,linear,1,scale,0.98
2,1,rbf,1,auto,0.98
3,1,linear,1,auto,0.98
5,1,linear,2,scale,0.98
6,1,rbf,2,auto,0.98
7,1,linear,2,auto,0.98
9,1,linear,3,scale,0.98
10,1,rbf,3,auto,0.98
11,1,linear,3,auto,0.98
12,10,rbf,1,scale,0.98


In [78]:
from sklearn.model_selection import RandomizedSearchCV

RS_cv = RandomizedSearchCV(SVC(),{
    'C': [1,10,20],
    'kernel': ['rbf','linear'],
    'degree': [1,2,3],
    'gamma': ['scale','auto']
}, cv = 5, return_train_score = False, n_iter=10)

RS_cv.fit(d.data, d.target)

RandomizedSearchCV(cv=5, estimator=SVC(),
                   param_distributions={'C': [1, 10, 20], 'degree': [1, 2, 3],
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['rbf', 'linear']})

In [79]:
RS_cv.best_score_

0.9800000000000001

In [82]:
hr3 = pd.DataFrame(RS_cv.cv_results_)
hr3.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_gamma,param_degree,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.0006,0.00049,0.0004,0.00049,rbf,auto,3,10,"{'kernel': 'rbf', 'gamma': 'auto', 'degree': 3...",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.0006,0.00049,0.0004,0.00049,rbf,auto,1,1,"{'kernel': 'rbf', 'gamma': 'auto', 'degree': 1...",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.0008,0.00040,0.0002,0.00040,rbf,scale,2,1,"{'kernel': 'rbf', 'gamma': 'scale', 'degree': ...",0.966667,0.966667,0.966667,0.933333,1.0,0.966667,0.021082,8
3,0.0002,0.00040,0.0006,0.00049,linear,auto,1,20,"{'kernel': 'linear', 'gamma': 'auto', 'degree'...",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,8
4,0.0008,0.00040,0.0002,0.00040,linear,scale,3,10,"{'kernel': 'linear', 'gamma': 'scale', 'degree...",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,5


In [83]:
hr4 = hr3[['param_C','param_kernel','param_degree','param_gamma','mean_test_score']]
hr4

,param_C,param_kernel,param_degree,param_gamma,mean_test_score
0,10,rbf,3,auto,0.980000
1,1,rbf,1,auto,0.980000
2,1,rbf,2,scale,0.966667
3,20,linear,1,auto,0.966667
4,10,linear,3,scale,0.973333
5,20,rbf,2,auto,0.966667
6,20,linear,2,auto,0.966667
7,20,rbf,1,auto,0.966667
8,1,linear,1,auto,0.980000
9,1,linear,1,scale,0.980000


In [84]:
hr4[hr4['mean_test_score'] == 0.9800000000000001]

,param_C,param_kernel,param_degree,param_gamma,mean_test_score
0,10,rbf,3,auto,0.98
1,1,rbf,1,auto,0.98
8,1,linear,1,auto,0.98
9,1,linear,1,scale,0.98


In [85]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [86]:
model = {
    'Support Vector': {
        'model': SVC(),
        'param': {
            'C': [1,10,20],
            'kernel': ['rbf','linear'],
            'degree': [1,2,3],
            'gamma': ['scale','auto']
        }
    },
    
    'Random Forest':{
        'model': RandomForestClassifier(),
        'param': {
            'n_estimators': [50,100,150],
            'random_state': [2,3,4]
        }
    },
    
    'Logistic Regression': {
        'model': LogisticRegression(),
        'param': {
            'C': [1,5,10],
            'random_state': [2,3,4]
        }
    }
}

In [87]:
from sklearn.model_selection import GridSearchCV

In [90]:
model.items()

dict_items([('Support Vector', {'model': SVC(), 'param': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear'], 'degree': [1, 2, 3], 'gamma': ['scale', 'auto']}}), ('Random Forest', {'model': RandomForestClassifier(), 'param': {'n_estimators': [50, 100, 150], 'random_state': [2, 3, 4]}}), ('Logistic Regression', {'model': LogisticRegression(), 'param': {'C': [1, 5, 10], 'random_state': [2, 3, 4]}})])

In [91]:
score = []
for model_name, i in model.items():
    clf = GridSearchCV(i['model'], i['param'], cv = 5, return_train_score = False)
    clf.fit(d.data, d.target)
    score.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
    

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

In [93]:
pd.DataFrame(score)

,model,best_score,best_params
0,Support Vector,0.980000,"{'C': 1, 'degree': 1, 'gamma': 'scale', 'kerne..."
1,Random Forest,0.966667,"{'n_estimators': 100, 'random_state': 4}"
2,Logistic Regression,0.980000,"{'C': 10, 'random_state': 2}"


In [94]:
score

[{'model': 'Support Vector',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 1, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}},
 {'model': 'Random Forest',
  'best_score': 0.9666666666666668,
  'best_params': {'n_estimators': 100, 'random_state': 4}},
 {'model': 'Logistic Regression',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 10, 'random_state': 2}}]

# Exercise: Machine Learning Finding Optimal Model and Hyperparameters
For digits dataset in sklearn.dataset, please try following classifiers and find out the one that gives best performance. Also find the optimal parameters for that classifier.

from sklearn import svm

from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB

from sklearn.naive_bayes import MultinomialNB

from sklearn.tree import DecisionTreeClassifier

In [95]:
from sklearn.datasets import load_digits
r = load_digits()

In [96]:
dir(r)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [97]:
pd.DataFrame(r.data, columns = r.feature_names)

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,0.0,0.0,4.0,10.0,13.0,6.0,0.0,0.0,0.0,1.0,...,4.0,0.0,0.0,0.0,2.0,14.0,15.0,9.0,0.0,0.0
1793,0.0,0.0,6.0,16.0,13.0,11.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,6.0,16.0,14.0,6.0,0.0,0.0
1794,0.0,0.0,1.0,11.0,15.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,9.0,13.0,6.0,0.0,0.0
1795,0.0,0.0,2.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,5.0,12.0,16.0,12.0,0.0,0.0


In [98]:
r.target_names

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [99]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [100]:
model1 = {
    'GaussianNB': {
        'model': GaussianNB(),
        'param': {
            
        }
    },
    
    'MultinomialNB': {
        'model': MultinomialNB(),
        'param': {
            'alpha': [1,0]
        }
    },
    
    'LogisticRegression': {
        'model': LogisticRegression(),
        'param': {
            'C': [1,5,10],
            'random_state': [2,3,4]
        }
    },
    
    'SupportVector(SVC)': {
        'model': SVC(),
        'param': {
            'C': [1,10,20],
            'kernel': ['rbf','linear'],
            'degree': [1,2,3],
            'gamma': ['scale','auto']
        }
    },
    
    'RandomForest': {
        'model': RandomForestClassifier(),
        'param': {
            'n_estimators': [50,100,150],
            'random_state': [2,3,4]
        }
    },
    
    'DecisionTree': {
        'model': DecisionTreeClassifier(),
        'param': {
            'criterion': ['gini', 'entropy', 'log_loss'],
            'splitter': ['best','random']
        }
    }
}

In [101]:
model1.items()

dict_items([('GaussianNB', {'model': GaussianNB(), 'param': {}}), ('MultinomialNB', {'model': MultinomialNB(), 'param': {'alpha': [1, 0]}}), ('LogisticRegression', {'model': LogisticRegression(), 'param': {'C': [1, 5, 10], 'random_state': [2, 3, 4]}}), ('SupportVector(SVC)', {'model': SVC(), 'param': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear'], 'degree': [1, 2, 3], 'gamma': ['scale', 'auto']}}), ('RandomForest', {'model': RandomForestClassifier(), 'param': {'n_estimators': [50, 100, 150], 'random_state': [2, 3, 4]}}), ('DecisionTree', {'model': DecisionTreeClassifier(), 'param': {'criterion': ['gini', 'entropy', 'log_loss'], 'splitter': ['best', 'random']}})])

In [102]:
from sklearn.model_selection import GridSearchCV

In [104]:
score1 = []

for model_names , i in model1.items():
    clf1 = GridSearchCV(i['model'], i['param'], cv = 5, return_train_score = False)
    clf1.fit(r.data, r.target)
    
    score1.append({
        'Model Names': model_names,
        'Best Score': clf1.best_score_,
        'Best Parameter': clf1.best_params_,
        'Best Estimetor': clf1.best_estimator_
    })
    
pd.DataFrame(score1)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The d

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

,Model Names,Best Score,Best Parameter,Best Estimetor
0,GaussianNB,0.806928,{},GaussianNB()
1,MultinomialNB,0.870350,{'alpha': 1},MultinomialNB(alpha=1)
2,LogisticRegression,0.913765,"{'C': 1, 'random_state': 2}","LogisticRegression(C=1, random_state=2)"
3,SupportVector(SVC),0.973850,"{'C': 10, 'degree': 1, 'gamma': 'scale', 'kern...","SVC(C=10, degree=1)"
4,RandomForest,0.938248,"{'n_estimators': 150, 'random_state': 3}","(DecisionTreeClassifier(max_features='sqrt', r..."
5,DecisionTree,0.808596,"{'criterion': 'log_loss', 'splitter': 'best'}",DecisionTreeClassifier(criterion='log_loss')


In [105]:
score1

[{'Model Names': 'GaussianNB',
  'Best Score': 0.8069281956050759,
  'Best Parameter': {},
  'Best Estimetor': GaussianNB()},
 {'Model Names': 'MultinomialNB',
  'Best Score': 0.8703497369235531,
  'Best Parameter': {'alpha': 1},
  'Best Estimetor': MultinomialNB(alpha=1)},
 {'Model Names': 'LogisticRegression',
  'Best Score': 0.9137650882079852,
  'Best Parameter': {'C': 1, 'random_state': 2},
  'Best Estimetor': LogisticRegression(C=1, random_state=2)},
 {'Model Names': 'SupportVector(SVC)',
  'Best Score': 0.9738502011761063,
  'Best Parameter': {'C': 10, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'},
  'Best Estimetor': SVC(C=10, degree=1)},
 {'Model Names': 'RandomForest',
  'Best Score': 0.938248220365212,
  'Best Parameter': {'n_estimators': 150, 'random_state': 3},
  'Best Estimetor': RandomForestClassifier(n_estimators=150, random_state=3)},
 {'Model Names': 'DecisionTree',
  'Best Score': 0.8085964097802538,
  'Best Parameter': {'criterion': 'log_loss', 'splitter': 'best'}